In [193]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.functions import *
from pyspark.sql.session import SparkSession
from splink import Splink
from splink.jar_fallback import jw_sim_py

In [194]:
print("PySpark Application Started")

spark = SparkSession \
    .builder \
    .appName("Splink") \
    .master("local[*]") \
    .config("spark.default.parallelism", "4")\
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.memory", "8g")\
    .getOrCreate()


print(spark.sparkContext.appName)

df = spark.read.option("header", "true").csv("/home/abhishekks446/data/demo_20.csv")
print("the data is:")

# df.createOrReplaceTempView("table1")
# df_new=spark.sql("select * from table1 ORDER BY RAND() limit 8000")
df_new=df

PySpark Application Started
Splink
the data is:


In [195]:
df_new.count()

20

In [196]:
settings = {
    "link_type": "dedupe_only",
    "blocking_rules": [
        "l.venue = r.venue and l.year = r.year",
        "l.venue = r.venue and l.title = r.title"

    ],
    "comparison_columns": [

        {
            "col_name": "title",
            "num_levels": 2,
            "term_frequency_adjustments": True
        },
#         {
#             "col_name": "year"

#         },
         {
            "col_name": "venue",
          
        }
       
    ]

}

linker = Splink(settings,df_new, spark)
df_e = linker.get_scored_comparisons()

/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/case_statements.py:20: UserWarning: 

Custom string comparison functions such as jaro_winkler_sim are not available in Spark
Or you did not pass 'spark' (the SparkSession) into 'Model' 
You can import these functions using the scala-udf-similarity-0.0.9.jar provided with Splink.
You will need to add it by correctly configuring your spark config
For example in Spark 2.4.5

from pyspark.sql import SparkSession, types
from pyspark.context import SparkConf, SparkContext
conf.set('spark.driver.extraClassPath', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar') # Not needed in spark 3
conf.set('spark.jars', '/home/abhishekks446/pyspark/my_project_dir/my_project_env/lib/python3.8/site-packages/splink/jars/scala-udf-similarity-0.0.9.jar')
spark.udf.registerJavaFunction('jaro_winkler_sim','uk.gov.moj.dash.linkage.JaroWinkle

In [197]:
# settings = {
#     "link_type": "dedupe_only",
#     "blocking_rules": [
#         "l.country = r.country and l.lat = r.lat"

#     ],
#     "comparison_columns": [

#         {
#             "col_name": "delta",
#             "num_levels": 2,
# #             "term_frequency_adjustments": True
#         },
#         {
#             "col_name": "lat"

#         },
#          {
#             "col_name": "country",
          
#         }
       
#     ]

# }

# linker = Splink(settings,df_new, spark)
# df_e = linker.get_scored_comparisons()

In [198]:
df_e.count()

4

In [202]:
df_e.coalesce(1).write.format("csv").option("header","true").mode("overwrite").save("/home/abhishekks446/data/output_num")

22/07/16 15:58:06 WARN TransportChannelHandler: Exception in connection from KLI0521/192.168.10.12:34971
java.io.IOException: Connection timed out
	at java.base/sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at java.base/sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at java.base/sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:276)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:233)
	at java.base/sun.nio.ch.IOUtil.read(IOUtil.java:223)
	at java.base/sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:356)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:253)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1133)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:350)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:148)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:714)
	at io.netty.channel.nio.NioEventLoop.processSelecte

In [200]:
df_new=df_e.toPandas().sort_values("match_probability", ascending=False)
df_new=df_new.sort_values("match_weight", ascending=False)

In [201]:
df_new.head(21)

,match_weight,match_probability,unique_id_l,unique_id_r,title_l,title_r,gamma_title,venue_l,venue_r,gamma_venue,match_key
1,1.716318,0.76668,8,9,Report on DART '96: Databases: Active and Real...,Report on DART '96: Databases: Active and Real...,1,SIGMOD Record,SIGMOD Record,1,0
0,-1.222392,0.30000,5,6,Efficient Geometry-based Similarity Search of ...,Efficient Geometry-3D Spatial Databases,0,SIGMOD Record,SIGMOD Record,1,0
2,-1.222392,0.30000,13,14,WaveCluster: A Wavelet Based Clustering Approa...,A Wavelet Based Clustering Approach for Spatia...,0,VLDB J.,VLDB J.,1,0
3,-1.222392,0.30000,16,17,Snowball: A Prototype System for Extracting Re...,Snowball: A Prototype,0,SIGMOD Conference,SIGMOD Conference,1,0


In [76]:
dedupe=df_new.loc[df_new['match_probability'] > 0.90]

In [77]:
dedupe.count()

match_weight         21
match_probability    21
unique_id_l          21
unique_id_r          21
title_l              21
title_r              21
gamma_title          21
year_l               21
year_r               21
gamma_year           21
venue_l              21
venue_r              21
gamma_venue          21
dtype: int64